# Solar Power Generation Prediction Inference (prediction testing)

* I preprocessed data first, loaded the trained model, and predicted using test data
* Used 1D-CNN model for time series (training code in training.ipynb)
* The model predicts the amount of power generation of 73th hour by using data of previous 72 hours

In [ ]:
#preparation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
from google.colab import drive
from datetime import datetime, timedelta

import math
import time
import random

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.externals import joblib

drive.mount('./gdrive', force_remount=True)
plt.style.use("fivethirtyeight")
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)
warnings.filterwarnings(action='ignore')

Mounted at ./gdrive


In [ ]:
dangjin_weather_06 = pd.read_csv("./sun/dangjin_weather_06.csv", encoding = 'utf-8')
ulsan_weather_06 = pd.read_csv("./sun/ulsan_weather_06.csv", encoding = 'utf-8')

# Preprocessing

In [ ]:
def interpolatee(dangjin_fcst):
  dangjin_fcst['Temperature'] = dangjin_fcst['Temperature'].interpolate() 
  dangjin_fcst['Humidity'] = dangjin_fcst['Humidity'].interpolate()
  dangjin_fcst['WindSpeed'] = dangjin_fcst['WindSpeed'].interpolate()
  dangjin_fcst['Cloud'] = dangjin_fcst['Cloud'].interpolate()
  return dangjin_fcst

In [ ]:
dangjin_weather_06 = interpolatee(dangjin_weather_06)
ulsan_weather_06 = interpolatee(ulsan_weather_06)

In [ ]:
dangjin_weather_06['WindDirection_x'] = dangjin_weather_06['WindDirection'].apply(angle_to_cos)  
dangjin_weather_06['WindDirection_y'] = dangjin_weather_06['WindDirection'].apply(angle_to_sin)
dangjin_weather_06['Wind_x'] = dangjin_weather_06['WindDirection_x'] * dangjin_weather_06['WindSpeed']
dangjin_weather_06['Wind_y'] = dangjin_weather_06['WindDirection_y'] * dangjin_weather_06['WindSpeed']

ulsan_weather_06['WindDirection_x'] = ulsan_weather_06['WindDirection'].apply(angle_to_cos)
ulsan_weather_06['WindDirection_y'] = ulsan_weather_06['WindDirection'].apply(angle_to_sin)
ulsan_weather_06['Wind_x'] = ulsan_weather_06['WindDirection_x'] * ulsan_weather_06['WindSpeed']
ulsan_weather_06['Wind_y'] = ulsan_weather_06['WindDirection_y'] * ulsan_weather_06['WindSpeed']

dangjin_weather_06['WindDirection'] = dangjin_weather_06['WindDirection'].interpolate()
ulsan_weather_06['WindDirection'] = ulsan_weather_06['WindDirection'].interpolate()

In [ ]:
dangjin_weather_06['Wind_x'] = dangjin_weather_06['Wind_x'].interpolate()
dangjin_weather_06['Wind_y'] = dangjin_weather_06['Wind_y'].interpolate()

ulsan_weather_06['Wind_x'] = ulsan_weather_06['Wind_x'].interpolate()
ulsan_weather_06['Wind_y'] = ulsan_weather_06['Wind_y'].interpolate()

In [ ]:
dangjin_weather_06['hour'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.hour
dangjin_weather_06['day'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.day
dangjin_weather_06['month'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.month

ulsan_weather_06['hour'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.hour
ulsan_weather_06['day'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.day
ulsan_weather_06['month'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.month

In [ ]:
#Add dewpoint by calculation
b = 17.62
c = 243.12
gamma = (b * dangjin_weather_06['Temperature'] /(c + dangjin_weather_06['Temperature'])) + np.log(dangjin_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
dangjin_weather_06['DewPoint'] = dewpoint  

gamma = (b *ulsan_weather_06['Temperature'] /(c + ulsan_weather_06['Temperature'])) + np.log(ulsan_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
ulsan_weather_06['DewPoint'] = dewpoint

In [ ]:
dangjin_weather_06['방위각'] = dangjin_weather_06['방위각'].apply(lambda x : int(x[:3]))
dangjin_weather_06['적경'] = dangjin_weather_06['적경'].apply(lambda x : int(x.split(" ")[0]))
dangjin_weather_06['고도'] = dangjin_weather_06['고도'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
dangjin_weather_06['적위'] = dangjin_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

ulsan_weather_06['방위각'] = ulsan_weather_06['방위각'].apply(lambda x : int(x[:3]))
ulsan_weather_06['고도'] = ulsan_weather_06['고도'].apply(lambda x : int(x.split(" ")[0]))
ulsan_weather_06['적경'] = ulsan_weather_06['적경'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
ulsan_weather_06['적위'] = ulsan_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

In [ ]:
using_col = ['Temperature', 'DewPoint', 'Humidity', 'Cloud', 'Wind_x', 'Wind_y',
             'hour', 'month', '방위각',	'고도',	'적경',	'적위','일조','일사']

dangjin_weather_06 = dangjin_weather_06[using_col].fillna(0)
ulsan_weather_06 = ulsan_weather_06[using_col].fillna(0)

In [ ]:
# Shift => like window sliding
# slide for 72 hours, each row contains latest 72 hours of data
def shift(df, period):
  target = pd.DataFrame()
  for i in range(period, -1, -1): 
    tmp = df.shift(i) #i= period
    target = pd.concat([target, tmp], axis = 1)
  return target

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,20.6,19.253190,92.0,4.0,1.221600,0.444626,1,7,5,-29.3646,6,23.0628,0.000000,0.000000
1,20.3,19.129697,93.0,3.0,1.221600,0.444626,2,7,21,-26.4803,6,23.0618,0.000000,0.000000
2,20.2,18.857262,92.0,1.0,0.689440,0.578509,3,7,34,-21.0902,6,23.0609,0.000000,0.000000
3,19.8,19.309399,97.0,7.0,0.563816,0.205212,4,7,46,-13.1800,6,23.0559,0.000000,0.000000
4,19.7,19.374642,98.0,5.0,0.171010,0.469846,5,7,57,-3.5213,6,23.0549,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,25.0,21.306887,80.0,1.0,-2.599296,-0.060499,19,7,290,9.1746,7,22.1615,0.835639,0.123938
211,24.0,21.320146,85.0,1.0,-2.097726,-0.097703,20,7,299,-1.3126,7,22.1556,0.190122,0.000000
212,23.0,21.269494,90.0,1.0,-1.596102,-0.111610,21,7,309,-11.2300,7,22.1537,0.000000,0.000000
213,23.0,21.269494,90.0,1.0,-1.596102,-0.111610,22,7,320,-19.4934,7,22.1518,0.000000,0.000000


In [ ]:
dangjin_weather_06 = shift(dangjin_weather_06, 71) 
ulsan_weather_06 = shift(ulsan_weather_06, 71) 

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,...,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.6,19.253190,92.0,4.0,1.221600,0.444626,1,7,5,-29.3646,6,23.0628,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.6,19.253190,92.0,4.0,1.221600,4.446262e-01,1.0,7.0,5.0,-29.3646,6.0,23.0628,0.000000,0.000000,20.3,19.129697,93.0,3.0,1.221600,0.444626,2,7,21,-26.4803,6,23.0618,0.000000,0.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.000000,4.000000,1.221600,4.446262e-01,1.0,7.0,5.0,-29.3646,6.0,23.0628,0.000000,0.000000,20.3,19.129697,93.0,3.0,1.221600,4.446262e-01,2.0,7.0,21.0,-26.4803,6.0,23.0618,0.000000,0.000000,20.2,18.857262,92.0,1.0,0.689440,0.578509,3,7,34,-21.0902,6,23.0609,0.000000,0.000000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,93.000000,3.000000,1.221600,4.446262e-01,2.0,7.0,21.0,-26.4803,6.0,23.0618,0.000000,0.000000,20.2,18.857262,92.0,1.0,0.689440,5.785088e-01,3.0,7.0,34.0,-21.0902,6.0,23.0609,0.000000,0.000000,19.8,19.309399,97.0,7.0,0.563816,0.205212,4,7,46,-13.1800,6,23.0559,0.000000,0.000000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.000000,1.000000,0.689440,5.785088e-01,3.0,7.0,34.0,-21.0902,6.0,23.0609,0.000000,0.000000,19.8,19.309399,97.0,7.0,0.563816,2.052121e-01,4.0,7.0,46.0,-13.1800,6.0,23.0559,0.000000,0.000000,19.7,19.374642,98.0,5.0,0.171010,0.469846,5,7,57,-3.5213,6,23.0549,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,23.3,22.965673,98.0,10.0,3.673940e-17,-0.600000,20.0,7.0,299.0,-1.2229,7.0,22.3636,0.0,0.03,23.4,23.065422,98.0,10.0,2.449294e-17,0.400000,21.0,7.0,309.0,-11.1143,7.0,22.3620,0.0,0.0,23.3,22.965673,98.0,10.0,3.830222e-01,0.321394,22.0,7.0,321.0,-19.3532,7.0,22.3604,...,73.333333,1.666667,-3.484848,3.253245e-01,17.0,7.0,274.0,32.3405,7.0,22.1654,0.949468,0.390278,26.0,21.222516,75.0,1.0,-3.100000,-7.592810e-16,18.0,7.0,282.0,20.4443,7.0,22.1634,0.913783,0.275528,25.0,21.306887,80.0,1.0,-2.599296,-0.060499,19,7,290,9.1746,7,22.1615,0.835639,0.123938
211,23.4,23.065422,98.0,10.0,2.449294e-17,0.400000,21.0,7.0,309.0,-11.1143,7.0,22.3620,0.0,0.00,23.3,22.965673,98.0,10.0,3.830222e-01,0.321394,22.0,7.0,321.0,-19.3532,7.0,22.3604,0.0,0.0,23.4,23.065422,98.0,10.0,1.469576e-16,-2.400000,23.0,7.0,334.0,-25.5807,7.0,22.3548,...,75.000000,1.000000,-3.100000,-7.592810e-16,18.0,7.0,282.0,20.4443,7.0,22.1634,0.913783,0.275528,25.0,21.306887,80.0,1.0,-2.599296,-6.049929e-02,19.0,7.0,290.0,9.1746,7.0,22.1615,0.835639,0.123938,24.0,21.320146,85.0,1.0,-2.097726,-0.097703,20,7,299,-1.3126,7,22.1556,0.190122,0.000000
212,23.3,22.96567

In [ ]:
#Delete the first 72 data as those can't contain all of latest 72 hours of data
dangjin_weather_06 = dangjin_weather_06[(dangjin_weather_06.isnull().sum(axis = 1) <= 0).values]
ulsan_weather_06 = ulsan_weather_06[(ulsan_weather_06.isnull().sum(axis = 1) <= 0).values]

'\ndangjin_weather_06 = dangjin_weather_06[(dangjin_weather_06.isnull().sum(axis = 1) <= 0).values]\nulsan_weather_06 = ulsan_weather_06[(ulsan_weather_06.isnull().sum(axis = 1) <= 0).values]\n'

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,...,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.6,19.253190,92.0,4.0,1.221600,0.444626,1,7,5,-29.3646,6,23.0628,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.6,19.253190,92.0,4.0,1.221600,4.446262e-01,1.0,7.0,5.0,-29.3646,6.0,23.0628,0.000000,0.000000,20.3,19.129697,93.0,3.0,1.221600,0.444626,2,7,21,-26.4803,6,23.0618,0.000000,0.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.000000,4.000000,1.221600,4.446262e-01,1.0,7.0,5.0,-29.3646,6.0,23.0628,0.000000,0.000000,20.3,19.129697,93.0,3.0,1.221600,4.446262e-01,2.0,7.0,21.0,-26.4803,6.0,23.0618,0.000000,0.000000,20.2,18.857262,92.0,1.0,0.689440,0.578509,3,7,34,-21.0902,6,23.0609,0.000000,0.000000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,93.000000,3.000000,1.221600,4.446262e-01,2.0,7.0,21.0,-26.4803,6.0,23.0618,0.000000,0.000000,20.2,18.857262,92.0,1.0,0.689440,5.785088e-01,3.0,7.0,34.0,-21.0902,6.0,23.0609,0.000000,0.000000,19.8,19.309399,97.0,7.0,0.563816,0.205212,4,7,46,-13.1800,6,23.0559,0.000000,0.000000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.000000,1.000000,0.689440,5.785088e-01,3.0,7.0,34.0,-21.0902,6.0,23.0609,0.000000,0.000000,19.8,19.309399,97.0,7.0,0.563816,2.052121e-01,4.0,7.0,46.0,-13.1800,6.0,23.0559,0.000000,0.000000,19.7,19.374642,98.0,5.0,0.171010,0.469846,5,7,57,-3.5213,6,23.0549,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,23.3,22.965673,98.0,10.0,3.673940e-17,-0.600000,20.0,7.0,299.0,-1.2229,7.0,22.3636,0.0,0.03,23.4,23.065422,98.0,10.0,2.449294e-17,0.400000,21.0,7.0,309.0,-11.1143,7.0,22.3620,0.0,0.0,23.3,22.965673,98.0,10.0,3.830222e-01,0.321394,22.0,7.0,321.0,-19.3532,7.0,22.3604,...,73.333333,1.666667,-3.484848,3.253245e-01,17.0,7.0,274.0,32.3405,7.0,22.1654,0.949468,0.390278,26.0,21.222516,75.0,1.0,-3.100000,-7.592810e-16,18.0,7.0,282.0,20.4443,7.0,22.1634,0.913783,0.275528,25.0,21.306887,80.0,1.0,-2.599296,-0.060499,19,7,290,9.1746,7,22.1615,0.835639,0.123938
211,23.4,23.065422,98.0,10.0,2.449294e-17,0.400000,21.0,7.0,309.0,-11.1143,7.0,22.3620,0.0,0.00,23.3,22.965673,98.0,10.0,3.830222e-01,0.321394,22.0,7.0,321.0,-19.3532,7.0,22.3604,0.0,0.0,23.4,23.065422,98.0,10.0,1.469576e-16,-2.400000,23.0,7.0,334.0,-25.5807,7.0,22.3548,...,75.000000,1.000000,-3.100000,-7.592810e-16,18.0,7.0,282.0,20.4443,7.0,22.1634,0.913783,0.275528,25.0,21.306887,80.0,1.0,-2.599296,-6.049929e-02,19.0,7.0,290.0,9.1746,7.0,22.1615,0.835639,0.123938,24.0,21.320146,85.0,1.0,-2.097726,-0.097703,20,7,299,-1.3126,7,22.1556,0.190122,0.000000
212,23.3,22.96567

In [ ]:
#min max scaling
dangjin_min = dangjin_weather_06.min()
dangjin_max = dangjin_weather_06.max()

dangjin_weather_06 = (dangjin_weather_06 - dangjin_min) / (dangjin_max - dangjin_min)

In [ ]:
ulsan_min = ulsan_weather_06.min()
ulsan_max = ulsan_weather_06.max()

ulsan_weather_06 = (ulsan_weather_06 - ulsan_min) / (ulsan_max - ulsan_min)

# Model

In [ ]:
import tensorflow as tf

In [ ]:
dangjin_weather_06.shape

(215, 1008)

In [ ]:
dangjin_weather_06 = dangjin_weather_06.values.reshape((dangjin_weather_06.shape[0], 72, 14, 1)) #(193, 864) -> (193, 72, 12, 1)
ulsan_weather_06 = ulsan_weather_06.values.reshape((ulsan_weather_06.shape[0], 72, 14, 1))

In [ ]:
dangjin_weather_06.shape

(215, 72, 14, 1)

In [ ]:
dangjin_weather_06 = tf.image.resize(dangjin_weather_06, (72, 32)) 
ulsan_weather_06 = tf.image.resize(ulsan_weather_06, (72, 32))

'\ndangjin_weather_06 = tf.image.resize(dangjin_weather_06, (72, 32)) \nulsan_weather_06 = tf.image.resize(ulsan_weather_06, (72, 32))\n'

In [ ]:
dangjin_weather_06 = np.squeeze(dangjin_weather_06, axis = -1) 
dangjin_weather_06 = np.stack((dangjin_weather_06,)*3, axis = -1)  

ulsan_weather_06 = np.squeeze(ulsan_weather_06, axis = -1)
ulsan_weather_06 = np.stack((ulsan_weather_06,)*3, axis = -1)

In [ ]:
dangjin_test = dangjin_weather_06[-24:]
ulsan_test = ulsan_weather_06[-24:]

* Inference

In [ ]:
floating_model0 = tf.keras.models.load_model("./model/floating_fold0.h5")
floating_model1 = tf.keras.models.load_model("./model/floating_fold1.h5")
floating_model2 = tf.keras.models.load_model("./model/floating_fold2.h5")
floating_model3 = tf.keras.models.load_model("./model/floating_fold3.h5")
floating_model4 = tf.keras.models.load_model("./model/floating_fold4.h5")

In [ ]:
floating0 = floating_model0.predict(dangjin_test)
floating1 = floating_model1.predict(dangjin_test)
floating2 = floating_model2.predict(dangjin_test)
floating3 = floating_model3.predict(dangjin_test)
floating4 = floating_model4.predict(dangjin_test)

In [ ]:
floating2

array([[  2.5831585],
       [  2.7795403],
       [  2.820041 ],
       [  2.8877432],
       [  2.8889914],
       [  2.817237 ],
       [  2.5082586],
       [  2.3272505],
       [  2.6325395],
       [  8.950737 ],
       [ 38.491158 ],
       [ 77.27165  ],
       [109.62871  ],
       [129.41975  ],
       [122.27581  ],
       [ 88.40043  ],
       [ 53.336605 ],
       [ 24.14802  ],
       [ 13.336593 ],
       [  6.648633 ],
       [  2.3461006],
       [  2.1167672],
       [  2.1440713],
       [  2.1962988]], dtype=float32)

In [ ]:
warehouse_model0 = tf.keras.models.load_model("./model/warehouse_fold0.h5")
warehouse_model1 = tf.keras.models.load_model("./model/warehouse_fold1.h5")
warehouse_model2 = tf.keras.models.load_model("./model/warehouse_fold2.h5")
warehouse_model3 = tf.keras.models.load_model("./model/warehouse_fold3.h5")
warehouse_model4 = tf.keras.models.load_model("./model/warehouse_fold4.h5")

In [ ]:
warehouse0 = warehouse_model0.predict(dangjin_test)
warehouse1 = warehouse_model1.predict(dangjin_test)
warehouse2 = warehouse_model2.predict(dangjin_test)
warehouse3 = warehouse_model3.predict(dangjin_test)
warehouse4 = warehouse_model4.predict(dangjin_test)

In [ ]:
dangjin_model0 = tf.keras.models.load_model("./model/dangjin_fold0.h5")
dangjin_model1 = tf.keras.models.load_model("./model/dangjin_fold1.h5")
dangjin_model2 = tf.keras.models.load_model("./model/dangjin_fold2.h5")
dangjin_model3 = tf.keras.models.load_model("./model/dangjin_fold3.h5")
dangjin_model4 = tf.keras.models.load_model("./model/dangjin_fold4.h5")

In [ ]:
dangjin0 = dangjin_model0.predict(dangjin_test)
dangjin1 = dangjin_model1.predict(dangjin_test)
dangjin2 = dangjin_model2.predict(dangjin_test)
dangjin3 = dangjin_model3.predict(dangjin_test)
dangjin4 = dangjin_model4.predict(dangjin_test)

In [ ]:
ulsan_model0 = tf.keras.models.load_model("./model/ulsan_fold0.h5")
ulsan_model1 = tf.keras.models.load_model("./model/ulsan_fold1.h5")
ulsan_model2 = tf.keras.models.load_model("./model/ulsan_fold2.h5")
ulsan_model3 = tf.keras.models.load_model("./model/ulsan_fold3.h5")
ulsan_model4 = tf.keras.models.load_model("./model/ulsan_fold4.h5")

In [ ]:
ulsan0 = ulsan_model0.predict(ulsan_test)
ulsan1 = ulsan_model1.predict(ulsan_test)
ulsan2 = ulsan_model2.predict(ulsan_test)
ulsan3 = ulsan_model3.predict(ulsan_test)
ulsan4 = ulsan_model4.predict(ulsan_test)

In [ ]:
floating = (floating0 + floating1 + floating2 + floating3 + floating4) / 5
warehouse = (warehouse0 + warehouse1 + warehouse2 + warehouse3 + warehouse4 ) / 5
dangjin = (dangjin0 + dangjin1 + dangjin2 + dangjin3 + dangjin4) / 5
ulsan = (ulsan0 + ulsan1 + ulsan2 + ulsan3 + ulsan4) / 5

In [ ]:
sample_submission = pd.read_csv("./validation.csv")

In [ ]:
sample_submission

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.0,0.000000,0.0,0.000000
1,2021-02-01 02:00:00,0.0,0.000000,0.0,0.000000
2,2021-02-01 03:00:00,0.0,0.000000,0.0,0.000000
3,2021-02-01 04:00:00,0.0,0.000000,0.0,0.000000
4,2021-02-01 05:00:00,0.0,0.000000,0.0,0.651203
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0.0,1.125614,0.0,2.495743
1388,2021-07-08 21:00:00,0.0,1.879234,0.0,3.404133
1389,2021-07-08 22:00:00,0.0,0.933403,0.0,4.150723
1390,2021-07-08 23:00:00,0.0,0.127186,0.0,3.220158


In [ ]:
sample_submission.loc[1367:1390]
sample_submission.tail()

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
1387,2021-07-08 20:00:00,0.0,1.125614,0.0,2.495743
1388,2021-07-08 21:00:00,0.0,1.879234,0.0,3.404133
1389,2021-07-08 22:00:00,0.0,0.933403,0.0,4.150723
1390,2021-07-08 23:00:00,0.0,0.127186,0.0,3.220158
1391,2021-07-08 24:00:00,0.0,2.028534,0.0,2.907734


In [ ]:
floating

array([[  1.16      ],
       [  0.90762854],
       [  0.8983763 ],
       [  0.8737954 ],
       [  0.9618743 ],
       [  1.0925175 ],
       [  1.278893  ],
       [  2.203419  ],
       [  8.800326  ],
       [ 83.40129   ],
       [203.69461   ],
       [288.14932   ],
       [325.84006   ],
       [351.4858    ],
       [371.68628   ],
       [353.27216   ],
       [254.43631   ],
       [129.92583   ],
       [ 42.58477   ],
       [  5.952838  ],
       [  1.4577174 ],
       [  0.98231256],
       [  0.9822527 ],
       [  1.150576  ]], dtype=float32)

In [ ]:
warehouse

array([[  1.3974291],
       [  1.2827843],
       [  1.3094455],
       [  1.4730264],
       [  1.6288216],
       [  1.7965695],
       [  1.9198654],
       [  2.1876276],
       [  5.764801 ],
       [ 35.865738 ],
       [ 92.64323  ],
       [135.16296  ],
       [160.7102   ],
       [181.89029  ],
       [197.8323   ],
       [188.43712  ],
       [134.08827  ],
       [ 53.87187  ],
       [  7.6001954],
       [  1.9505243],
       [  1.6565793],
       [  1.4682633],
       [  1.397657 ],
       [  1.4273384]], dtype=float32)

In [ ]:
floating.shape

(24, 1)

In [ ]:
sample_submission.loc[1367:1390].shape

(24, 5)

In [ ]:
sample_submission.loc[1367:1390, 'dangjin_floating'] = floating
sample_submission.loc[1367:1390, 'dangjin_warehouse'] = warehouse
sample_submission.loc[1367:1390, 'dangjin'] = dangjin
sample_submission.loc[1367:1390, 'ulsan'] = ulsan

In [ ]:
sample_submission.loc[1367:1390]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
1367,2021-07-07 24:00:00,1.160000,1.397429,6.419974,126.312035
1368,2021-07-08 01:00:00,0.907629,1.282784,5.451995,126.312035
1369,2021-07-08 02:00:00,0.898376,1.309446,4.597619,126.312035
1370,2021-07-08 03:00:00,0.873795,1.473026,4.277077,126.312035
1371,2021-07-08 04:00:00,0.961874,1.628822,3.997050,126.312035
1372,2021-07-08 05:00:00,1.092517,1.796569,4.765158,126.312035
1373,2021-07-08 06:00:00,1.278893,1.919865,4.861943,126.312035
1374,2021-07-08 07:00:00,2.203419,2.187628,6.048465,126.312035
1375,2021-07-08 08:00:00,8.800326,5.764801,10.944567,126.312035
1376,2021-07-08 09:00:00,83.401291,35.865738,102.678207,126.312035


In [ ]:
sample_submission.to_csv("final_inference.csv", index = False)